In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics
from scipy import stats
from scipy.stats import skew 
from sklearn.model_selection import train_test_split

import time

In [2]:
#importing the csv dataset
train = pd.read_csv('train.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# defining variables


In [43]:
features = train.copy()
features.drop(['Id', 'SalePrice'], axis=1, inplace=True)

y = train['SalePrice'][:,None]

### ohe

In [110]:
categorical = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond','Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
features = pd.get_dummies(features, columns= categorical, prefix= categorical)
features.head()

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,65.0,8450,2003,2003,196.0,706,0,150,856,856,...,0,0,0,1,0,0,0,0,1,0
1,80.0,9600,1976,1976,0.0,978,0,284,1262,1262,...,0,0,0,1,0,0,0,0,1,0
2,68.0,11250,2001,2002,162.0,486,0,434,920,920,...,0,0,0,1,0,0,0,0,1,0
3,60.0,9550,1915,1970,0.0,216,0,540,756,961,...,0,0,0,1,1,0,0,0,0,0
4,84.0,14260,2000,2000,350.0,655,0,490,1145,1145,...,0,0,0,1,0,0,0,0,1,0


In [5]:
n = features.shape[0]
d = features.shape[1]

# Normal eqn functions

In [9]:
def f_pred(X,w):
    f = X.dot(w)
    return f 

In [10]:
def loss_normaleqn(X,w):
    n = len(X)
    Xwy = X.dot(w)-y
    loss= 1/n*Xwy.T.dot(Xwy)
    return loss

In [11]:
# not sure if this is needed tbh

def grad_loss_normaleqn(y_pred,y,X):
    n = len(y)
    grad = 2/n*(X.T.dot(y_pred)-X.T.dot(y))
    return grad

In [12]:
def w_normaleqn(X,y):
    a = X.T.dot(X)
    b = X.T.dot(y)
    w = np.linalg.inv(a).dot(b)
    return w
    
# numpy. linalg. inv()

# For linear regression


In [34]:
# # ones = np.ones(n)
# X = features.to_numpy()
# X = np.insert(X, 0, 1, axis=1)

# w_init = np.ones(d)[:,None]

Try all features with standard ohe


In [113]:
X = features.copy()
X.insert(0, 'ones', 1)
X = X.to_numpy().astype(int)

In [114]:
X

array([[    1,    65,  8450, ...,     0,     1,     0],
       [    1,    80,  9600, ...,     0,     1,     0],
       [    1,    68, 11250, ...,     0,     1,     0],
       ...,
       [    1,    66,  9042, ...,     0,     1,     0],
       [    1,    68,  9717, ...,     0,     1,     0],
       [    1,    75,  9937, ...,     0,     1,     0]])

In [116]:
start = time.time()
w = w_normaleqn(X,y)

print('Time=',time.time() - start)
print(w)

Time= 0.0967409610748291
[[-2.95964576e+16]
 [ 2.34954471e+00]
 [ 1.91725958e+05]
 [-4.42981202e+07]
 [ 4.33427131e+07]
 [-4.13416351e+05]
 [-2.11001018e+06]
 [ 9.24009135e+06]
 [-2.46505049e+05]
 [ 3.72978702e+03]
 [-1.22717434e+04]
 [-2.79606815e+05]
 [ 3.49217495e+06]
 [-1.40683962e+04]
 [ 1.03573818e+08]
 [ 2.05346632e+08]
 [ 2.23310578e+08]
 [ 9.61602277e+07]
 [ 1.42762466e+08]
 [ 2.51669726e+08]
 [-4.58962151e+07]
 [ 1.62089292e+08]
 [-1.22378912e+01]
 [-2.34478747e+08]
 [ 5.34672639e+05]
 [-5.52144950e+04]
 [-1.33035538e+04]
 [ 4.71496387e+04]
 [-1.24533048e+05]
 [-8.20462612e+04]
 [ 4.40946065e+04]
 [-2.83143490e+04]
 [-7.16102827e+05]
 [ 5.29432388e+05]
 [ 1.59670101e+16]
 [ 1.59670101e+16]
 [ 1.59670095e+16]
 [ 1.59670107e+16]
 [ 1.59670102e+16]
 [ 1.59670105e+16]
 [ 1.59670103e+16]
 [ 1.59670105e+16]
 [ 1.59670101e+16]
 [ 1.59670100e+16]
 [ 2.23247659e+22]
 [ 1.59670095e+16]
 [ 1.59670098e+16]
 [ 1.59670094e+16]
 [ 1.59670088e+16]
 [-2.75183746e+15]
 [-2.75183740e+15]
 [-2.7

Try all features with modified ohe

In [67]:
# MSSubClass, MSZoning, Street, Alley, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, OverallQual, OverallCond, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, ExterQual, ExterCond,Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Heating, HeatingQC, CentralAir, Electrical, KitchenQual, Functional, FireplaceQu, GarageType, GarageFinish, GarageQual, GarageCond, PavedDrive, PoolQC, Fence, MiscFeature, SaleType, SaleCondition

In [17]:
cat = ['MSSubClass_20', 'MSZoning_FV', 'Street_Grv1', 'Alley_Grv1', 'LotShape_IR1', 
       'LandContour_Bnk', 'Utilities_AllPub', 'LotConfig_Corner', 'LandSlope_Gt1', 
       'Neighborhood_Blmngtn', 'Condition1_Artery', 'Condition2_Artery', 'BldgType_1Fam', 
       'HouseStyle_1.5Fin', 'OverallQual_1', 'OverallCond_1', 'RoofStyle_Flat', 'RoofMatl_ClyTile',
       'Exterior1st_AsbShng', 'Exterior2nd_AsbShng', 'MasVnrType_BrkCmn', 'ExterQual_Ex', 
       'ExterCond_Ex', 'Foundation_BrkTil', 'BsmtQual_Ex', 'BsmtCond_Fa', 'BsmtExposure_Av', 
       'BsmtFinType1_ALQ', 'BsmtFinType2_ALQ', 'Heating_Floor', 'HeatingQC_Ex', 'CentralAir_N', 
       'Electrical_FuseA', 'KitchenQual_Ex', 'Functional_Maj1', 'FireplaceQu_Ex', 
       'GarageType_2Types', 'GarageFinish_Fin', 'GarageQual_Ex', 'GarageCond_Ex', 'PavedDrive_N', 
       'PoolQC_Ex', 'Fence_GdPrv', 'MiscFeature_Gar2', 'SaleType_COD', 'SaleCondition_Abnorml']

one = np.ones(n).astype(int)
for i in cat:
    X[i] = one

Try only continuous data

In [101]:
drop = ['Id', 'SalePrice', 'MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond','Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']

X1 = train.copy()
X1.drop(drop, axis=1, inplace=True)
X1.insert(0, 'ones', 1)
X1=X1.to_numpy().astype(int)

In [103]:
X1.shape


(1460, 34)

Try only 10 continuous data


In [98]:
# sublist = ['LotArea', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2',
#            'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF']

sublist = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 
           'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF']

X2 = train[sublist]
X2.insert(0, 'ones', 1)
X2 = X2.to_numpy().astype(int)
X2.shape

(1460, 11)

In [100]:
start = time.time()
w = w_normaleqn(X2,y)

print('Time=',time.time() - start)
print(w)

Time= 0.0
[[ 4.40687413e+07]
 [ 1.98999224e+02]
 [-3.81710900e+04]
 [-2.15899272e+06]
 [ 2.15919172e+06]
 [ 5.96045137e+05]
 [ 1.82316677e+06]
 [-8.18319007e+05]
 [-1.30322950e+06]
 [ 1.23773146e+05]
 [ 3.41170611e+05]]


In [13]:
loss = loss_normaleqn(X1,w)
print(loss)

nan


In [99]:
a = X2.T.dot(X2)
# b = X.T.dot(y)
# w = np.linalg.inv(a).dot(b)

print(a)
# print(b)
# print(w)

[[       1460 -2147399518    15354569     2878051     2897904      150551
       647714       67962      828171     1543847     1697435]
 [-2147399518     6601002 -1211849480   165899055 -1980414733     9635089
  -2108391671     3840217 -2096167577    94248265 -2044878144]
 [   15354569 -1211849480  1892507273   209454323   416132615  1855251379
   -355963968   975908929   102948918   722893879 -1937238016]
 [    2878051   165899055   209454323  1379772913  1418116927   299257959
   1281832381   133622217  1635448846 -1244063852  -944064585]
 [    2897904 -1980414733   416132615  1418116927  1457604984   299784536
   1287390045   134566194  1646219094 -1226791963  -922987561]
 [     150551     9635089  1855251379   299257959   299784536    63180913
  -2049272717     3973840    98665216 -1946633661   209678350]
 [     647714 -2108391671  -355963968  1281832381  1287390045 -2049272717
    590860820    24770568   221786264   837417652   867749414]
 [      67962     3840217   975908929   1